<a href="https://colab.research.google.com/github/FaizaAli-Dev/AI-DS-Projects/blob/All-about-AI-and-Data-Science/CNN_Module_Project_FED_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import os
#Check the contents of the root directory in Google Drive
drive_root = '/content/drive/MyDrive/Computer_Vision'
for file_name in os.listdir(drive_root):
    print(file_name)

facial_expression.zip
Dataset


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Convolution2D,MaxPooling2D,Dense,Dropout,Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import cv2

In [ ]:
# Set the dataset path
dataset_path = '/content/drive/MyDrive/Computer_Vision/facial_expression.zip'
train_dir = '/content/drive/MyDrive/Computer_Vision/Dataset/train'
test_dir = '/content/drive/MyDrive/Computer_Vision/Dataset/test'

#Preprocessing of images during the training and validation

In [ ]:
validation_image_data_generator = ImageDataGenerator(rescale=1./255)
#"ImageDataGenerator" real-time data augmentation during model training
#1./255 means that each pixel value in the images will be divided by 255

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#Data augmentation paramerters setting
data_augumantation = ImageDataGenerator(
    rotation_range=10,      #image rotation degree
    width_shift_range=0.1,  #horizontal shift
    height_shift_range=0.1, #vertical shift
    shear_range=0.1,        #clockwise direction as radians
    zoom_range=0.1,         #zoom range
    horizontal_flip=True,   #flip images horizontally
    fill_mode='nearest'   #pixel value scale [0,1]
)

In [ ]:
batch_size = 128
#size of the image: 48*48 pixels
img_size = 48
training_set = data_augumantation.flow_from_directory('/content/drive/MyDrive/Computer_Vision/Dataset/train',
                                                      target_size=(img_size,img_size),
                                                      color_mode="grayscale",
                                                      batch_size=batch_size,
                                                      class_mode='categorical',
                                                      shuffle=True)

Test_set = data_augumantation.flow_from_directory('/content/drive/MyDrive/Computer_Vision/Dataset/test',
                                                  target_size=(img_size,img_size),
                                                  color_mode="grayscale",
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=False)

Found 32298 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [ ]:
# data cleaning
classes = os.listdir('/content/drive/MyDrive/Computer_Vision/Dataset/train')
images = []

clean_images = []
for i in range(len(images)):
    if os.path.exists(image_paths[i]):
        clean_images.append(images[i])

images = clean_images


In [ ]:
training_set.class_indices

{'anger': 0,
 'disgust': 1,
 'fear': 2,
 'happiness': 3,
 'neutral': 4,
 'sadness': 5,
 'surprise': 6}

#Creating FED Model using Sequential Model
I have add 2 convolution layers,dense layer and 1 fully connected layer


In [ ]:
from keras.layers import Conv2D,Input, MaxPooling2D, Dropout, Flatten, Dense, Activation
from keras.models import Model, Sequential
from keras.optimizers import Adam

# no of labels
total_classes= 7

# Initialising the CNN
FEM_model = Sequential()

#1st CNN layer
FEM_model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(48, 48,1))) #1
FEM_model.add(MaxPooling2D(pool_size=(2, 2)))
#2nd CNN layer
FEM_model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
FEM_model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten layer
FEM_model.add(Flatten())

#Fully connected 1st layer
FEM_model.add(Dense(256,activation = 'relu'))
#Fully connected layer 2nd layer
FEM_model.add(Dense(512,activation = 'relu'))

FEM_model.add(Dense(total_classes, activation='softmax'))

opt = Adam(lr=0.0001)
FEM_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
print(FEM_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 256)               2

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
emotion_model_info = FEM_model.fit(
        training_set ,
        steps_per_epoch=32298 // 64,
        epochs=10,
        validation_data=Test_set,
        validation_steps=3589 // 64)

Epoch 1/10
172/504 [=========>....................] - ETA: 4:46:13 - loss: 2.8003 - accuracy: 0.2191

In [ ]:
#from keras.callbacks import ModelCheckpoint
#checkpoint = ModelCheckpoint('/content/drive/My Drive/Colab Notebooks/checkpoint-{epoch:02d}.h5',
#                            save_weights_only=False,
#                            save_best_only=False,
#                             period=5)  # Save the model every 5 epochs
#
#
#FEM_model.fit(train_generator , validation_generator, epochs=100, callbacks=[checkpoint])

ValueError: ignored